In [2]:
!pip install eli5
!pip install --upgrade tables

     |████████████████████████████████| 112kB 1.4MB/s 
     |████████████████████████████████| 4.3MB 1.4MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4


In [0]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

## Wczytywanie danych

In [7]:
cd "/content/drive/My Drive/Colab Notebooks/matrix_two/dw_matrix_car"

/content/drive/My Drive/Colab Notebooks/matrix_two/dw_matrix_car


In [9]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

In [10]:
df.columns

Index(['breadcrumb', 'created_at', 'price_currency', 'price_details',
       'price_value', 'seller_address', 'seller_name', 'seller_type',
       'feature_czujniki-parkowania-przednie',
       'feature_poduszka-powietrzna-chroniąca-kolana',
       ...
       'param_pearl', 'param_stan', 'param_wersja', 'param_emisja-co2',
       'param_body-type', 'param_matowy', 'param_bezwypadkowy',
       'param_akryl-(niemetalizowany)', 'param_monthly-payment-value',
       'car_id'],
      dtype='object', length=155)

## Dummy Model

In [11]:
df.select_dtypes(np.number).columns

Index(['price_value', 'car_id'], dtype='object')

In [13]:
feats = ['car_id']
X = df[feats].values
y = df['price_value'].values

model = DummyRegressor()
model.fit(X,y)
y_pred = model.predict(X)

mean_absolute_error(y,y_pred)

39465.934630440985

In [14]:
[x for x in df.columns if 'price' in x]

['price_currency', 'price_details', 'price_value']

In [19]:
df['price_currency'].value_counts()

PLN    106290
Name: price_currency, dtype: int64

In [20]:
df = df[df['price_currency'] != 'EUR']
df.shape

(106290, 155)

## Features

In [0]:
SUFFIX_CAT='__cat'
for feat in df.columns:
  if isinstance(df[feat][0],list):continue

  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[feat + SUFFIX_CAT] = factorized_values

In [27]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [28]:
cat_feats

['created_at__cat',
 'seller_address__cat',
 'seller_name__cat',
 'seller_type__cat',
 'feature_czujniki-parkowania-przednie__cat',
 'feature_poduszka-powietrzna-chroniąca-kolana__cat',
 'feature_kurtyny-powietrzne__cat',
 'feature_klimatyzacja-dwustrefowa__cat',
 'feature_światła-led__cat',
 'feature_czujnik-zmierzchu__cat',
 'feature_elektrycznie-ustawiane-lusterka__cat',
 'feature_asr-(kontrola-trakcji)__cat',
 'feature_poduszka-powietrzna-kierowcy__cat',
 'feature_cd__cat',
 'feature_elektryczne-szyby-przednie__cat',
 'feature_poduszka-powietrzna-pasażera__cat',
 'feature_system-start-stop__cat',
 'feature_światła-do-jazdy-dziennej__cat',
 'feature_komputer-pokładowy__cat',
 'feature_elektryczne-szyby-tylne__cat',
 'feature_klimatyzacja-manualna__cat',
 'feature_tapicerka-welurowa__cat',
 'feature_czujnik-deszczu__cat',
 'feature_światła-przeciwmgielne__cat',
 'feature_ogrzewanie-postojowe__cat',
 'feature_radio-niefabryczne__cat',
 'feature_regulowane-zawieszenie__cat',
 'feature_

In [32]:
X = df[cat_feats].values
y = df['price_value']

model = DecisionTreeRegressor(max_depth = 5)

scores = cross_val_score(model, X,y,cv=3,scoring='neg_mean_absolute_error')
np.mean(scores)

-19566.588937368328

In [34]:
m = DecisionTreeRegressor(max_depth=5)
m.fit(X,y)

imp = PermutationImportance(m,random_state=325).fit(X,y)
eli5.show_weights(imp,feature_names=cat_feats)

Weight,Feature
0.2532 ± 0.0057,param_napęd__cat
0.1984 ± 0.0050,param_faktura-vat__cat
0.1957 ± 0.0054,param_stan__cat
0.1398 ± 0.0026,param_rok-produkcji__cat
0.0645 ± 0.0039,param_moc__cat
0.0415 ± 0.0013,feature_kamera-cofania__cat
0.0411 ± 0.0028,param_skrzynia-biegów__cat
0.0283 ± 0.0030,param_marka-pojazdu__cat
0.0199 ± 0.0015,param_pojemność-skokowa__cat
0.0162 ± 0.0005,feature_bluetooth__cat


In [0]:
  !git config --global user.email "nickypolit2@gmail.com"
  !git config --global user.name "Nikita"

In [0]:
!git add day2_visualization.ipynb
!git commit -m "add visualization v3"